# Bài thực hành số 3.1

Chào mừng đến với bài thực hành số 3.1. Trong bài thực hành này ta sẽ học cách sử dụng thuật toán DeepSORT với model YOLOv5 để tracking vật thể trong video. 

## Tổng quan

DeepSORT (Simple Online and Realtime Tracking) là một thuật toán theo dõi đối tượng trực tuyến mạnh mẽ, được phát triển dựa trên thuật toán SORT. DeepSORT cải tiến SORT bằng cách kết hợp các tính năng học sâu để theo dõi các đối tượng trong video với độ chính xác cao hơn, đặc biệt là trong các tình huống phức tạp như có nhiều đối tượng và các đối tượng xuất hiện hoặc biến mất khỏi khung hình.

DeepSORT được giới thiệu lần đầu tiên vào năm 2017 trong bài báo có tên "Simple Online and Realtime Tracking with a Deep Association Metric" bởi các tác giả là Wojke, Bewley, và Paulus. Sự cải tiến chính của DeepSORT so với SORT là việc sử dụng mạng nơ-ron sâu để tạo ra các vector đặc trưng giúp phân biệt giữa các đối tượng khác nhau, ngay cả khi có sự thay đổi về vị trí hay hình dạng.

Trong lĩnh vực xe tự hành (autonomous vehicles), DeepSORT có thể được sử dụng trong các hệ thống theo dõi đối tượng để nhận diện và theo dõi các phương tiện khác, người đi bộ, và các chướng ngại vật trên đường. Khả năng theo dõi liên tục và chính xác của DeepSORT giúp xe tự hành có thể đưa ra quyết định di chuyển an toàn hơn, tránh va chạm và đảm bảo lộ trình di chuyển được duy trì một cách hiệu quả.

## Mục tiêu học tập
Sau khi hoàn thành bài thực hành này, học viên sẽ học được các kiến thức:
  - DeepSORT

## Những kiến thức liên quan
  - Python
  - DeepSORT

## Nền tảng cần thiết
Để thực hiện được bài thực hành, bạn sẽ cần có những kiến thức sau:
  - Kĩ năng lập trình cơ bản với Python

## Bài toán
**Mục tiêu**: Áp dụng DeepSORT với model YOLO để theo dõi vật thể trong video

**Yêu cầu**
- Input: video
- Output: video có các ô theo dõi vật thể

## Hướng dẫn

Dưới đây là hướng dẫn chi tiết các bước để bạn có thể hiểu được quá trình áp dụng DeepSORT và YOLO lên video.


### Cài đặt thư viện cần thiết

In [21]:
import numpy as np
import torch
import cv2
import math
import time
from PIL import Image
from deep_sort_realtime.deepsort_tracker import DeepSort
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

### Tải model nhận diện

Việc các vật thể nào được theo dõi sẽ phụ thuộc vào việc model có khả năng nhận diện được hay không. Ở các bài lab chương 2, chúng tôi đã hướng dẫn bạn cách để có thể train được model hiệu quả cho từng mục đích. Hãy tải lại weights model bạn thấy phù hợp để hoàn thành bài tập sau.

Bài tập 1: Hoàn thành đoạn code sau bằng việc điền vào [...]

In [ ]:
# force reload: tránh xung đột tham số khi load model mới
model = torch.hub.load('ultralytics/yolov5', [...], path= [...], force_reload = [...])

### Khởi tạo DeepSORT
Ý nghĩa các tham số được sử dụng trong hàm `DeepSort`:

1. **`max_age`**:
   - **Ý nghĩa**: Xác định số khung hình tối đa mà một đối tượng có thể bị mất dấu trước khi bị loại khỏi bộ theo dõi. 
   - **Giải thích**: Nếu một đối tượng không được phát hiện trong `max_age` khung hình liên tiếp, nó sẽ bị coi là mất dấu và bị loại bỏ.

2. **`n_init`**:
   - **Ý nghĩa**: Xác định số lượng khung hình ban đầu mà một đối tượng cần phải được phát hiện liên tiếp trước khi nó được coi là một đối tượng theo dõi hợp lệ.
   - **Giải thích**: Một đối tượng mới phải xuất hiện trong ít nhất `n_init` khung hình trước khi nó được xác nhận là một đối tượng hợp lệ và bắt đầu được theo dõi.

3. **`nms_max_overlap`**:
   - **Ý nghĩa**: Tham số này liên quan đến Non-Maximum Suppression (NMS), một thuật toán được sử dụng để loại bỏ các khung giới hạn (bounding box) chồng lấp nhau với các đối tượng khác nhau.
   - **Giải thích**: `nms_max_overlap` xác định mức độ chồng lấp tối đa giữa các khung giới hạn mà vẫn được coi là hai đối tượng riêng biệt. Giá trị mặc định là `1.0`, có nghĩa là các khung giới hạn có thể chồng lấp hoàn toàn mà không bị loại bỏ.

4. **`max_cosine_distance`**:
   - **Ý nghĩa**: Xác định khoảng cách cosine tối đa giữa các đặc trưng của các đối tượng (ví dụ như embedding vectors) để có thể coi chúng là cùng một đối tượng.
   - **Giải thích**: Khoảng cách cosine là một thước đo sự tương tự giữa hai vectơ. Giá trị `max_cosine_distance` thấp cho thấy yêu cầu cao về sự tương tự giữa hai đối tượng để coi chúng là cùng một đối tượng. Giá trị mặc định thường là `0.3`, có nghĩa là chỉ các đối tượng có sự tương đồng cao mới được ghép nối.

Các tham số này có thể được điều chỉnh dựa trên dữ liệu và bài toán cụ thể mà bạn đang làm việc để tối ưu hóa hiệu suất của hệ thống theo dõi đối tượng.

Bài tập 2: Hoàn thành đoạn code sau bằng cách điền vào [...]

In [ ]:
# Khởi tạo DeepSORT
object_tracker = DeepSort(max_age=[...],
                          n_init=[...],
                          nms_max_overlap=[...],
                          max_cosine_distance=[...])

### Các hàm xử lý đầu vào cho thuật toán DeepSORT

Hàm `score_frame(frame)`: Hàm này nhận một khung hình (frame) làm đầu vào và trả về nhãn và tọa độ của các đối tượng được phát hiện trong khung hình.

Biến `classes`: là một danh sách hoặc từ điển chứa các tên lớp của mô hình YOLO, với chỉ số của lớp tương ứng với tên của đối tượng.
  
Hàm `class_to_label(x)`: Hàm này nhận vào chỉ số của lớp (x) và trả về tên lớp tương ứng.

Hàm `plot_boxes(results, height, width, confidence=0.3)`: Hàm này xử lý kết quả dự đoán của mô hình, lọc các đối tượng dựa trên độ tin cậy (confidence) và chuẩn bị thông tin về các hộp giới hạn (bounding boxes) để vẽ lên khung hình.

Bài tập 3: Hoàn thành đoạn code sau bằng cách điền vào [...]

In [ ]:
# Viết hàm để nhận khung hình và trả về các nhãn lớp và tọa độ của đối tượng
def score_frame(frame):
	results = [...]
	labels, cord = [...]
	return [...]

classes = model.names

# Dựa trên chỉ số lớp, trả về tên lớp tương ứng
def class_to_label(x):
	return [...]

# Hoàn thiện hàm để lọc và chuẩn bị thông tin về các đối tượng phát hiện được
def plot_boxes(results, height, width, confidence=0.3):
	labels, cord = [...]
	detections = []

	for i in range(len(labels)):
		row = cord[i]
		if [...]:
			x1, y1, x2, y2 = [...]
			conf = [...]
			class_label = [...]
			# print(feature)
			detections.append(
				([x1, y1, int(x2-x1), int(y2-y1)], conf, class_label))

	return detections

### Áp dụng DeepSORT và YOLO vào video

Bài tập 4: Hoàn thành code sau bằng việc điền vào [...]

In [ ]:
import cv2

input_video_path = [...] # Đường dẫn đến video đầu vào
output_video_path = [...]  # Đường dẫn đến video đầu ra

# Mở video đầu vào
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Lấy thông số của video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Tạo video writer để lưu video đầu ra
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec cho định dạng MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
while cap.isOpened():
	ret, frame = cap.read()
	if not ret:
		break

	# Dự đoán các đối tượng trong khung hình

	results = score_frame([...])
	detections = plot_boxes([...])
	tracks = object_tracker.update_tracks(detections, frame=frame)

	# Vẽ các khung hình và các ID theo dõi lên video
	for track in tracks:
		bbox = track.to_tlbr()  # Bounding box dưới dạng (x1, y1, x2, y2)
		track_id = track.track_id  # ID theo dõi
		x1, y1, x2, y2 = map(int, bbox)
		cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
		cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


	# Ghi khung hình vào video đầu ra
	out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
